In [1]:
import pandas as pd               # for data manipulation
import numpy as np                # for aggregation functions
import gc                         # for clean-up memory

In [2]:
orders = pd.read_csv('../dataset/orders.csv')
order_products_prior = pd.read_csv('../dataset/order_products__prior.csv')

# Get the product tables also --> To create metrics for aisles / departments 
products = pd.read_csv('../dataset/products.csv')
products.product_name = products.product_name.str.replace(' ', '_').str.lower()
products.product_name = products.product_name.str.replace(',', '-').str.lower()

In [3]:

# Create a DataFrame with the orders and the products that have been purchased on prior orders

,product_id,product_name,aisle_id,department_id
0,1,chocolate_sandwich_cookies,61,19
1,2,all-seasons_salt,104,13
2,3,robust_golden_unsweetened_oolong_tea,94,7
3,4,smart_ones_classic_favorites_mini_rigatoni_wit...,38,1
4,5,green_chile_anytime_sauce,5,13
...,...,...,...,...
49683,49684,vodka-_triple_distilled-_twist_of_vanilla,124,5
49684,49685,en_croute_roast_hazelnut_cranberry,42,1
49685,49686,artisan_baguette,112,3
49686,49687,smartblend_healthy_metabolism_dry_cat_food,41,8
